In [1]:
import numpy as np

# Chapter 2-- Image Formation

In [ ]:
def two_d_point(x,y):
    return np.array([x,y])

def homogeneous_to_inhomogeneous(x,y,w):
    augmented_vector=np.array([x,y,1])
    return w*augmented_vector

def point_on_two_d_line(x,y,a,b,c):
    x=np.array([x,y,1])
    l=np.array([a,b,c])
    return np.dot(x,l)

def normalize_l(l):
    norm=np.sqrt(np.dot(l,l))
    l_normalized=l/norm
    return l_normalized

def intersection_of_two_lines(a_1,b_1,c_1,a_2,b_2,c_2):
    first_line=np.array([a_1,b_1,c_1])
    second_line=np.array([a_2,b_2,c_2])
    x=np.cross(first_line,second_line)
    return x

def joining_two_points(x_1,y_1,x_2,y_2):
    first_point=np.array([x_1,y_1,1])
    second_point=np.array([x_2,y_2,1])
    l=np.cross(first_point,second_point)
    return l


    

# Vispy examples 

In [2]:
import sys

from vispy import app, gloo
from vispy.visuals import CubeVisual, transforms


class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, 'Cube', keys='interactive',
                            size=(400, 400))

        self.cube = CubeVisual((1.0, 0.5, 0.25), color='red', edge_color="k")
        self.theta = 0
        self.phi = 0

        # Create a TransformSystem that will tell the visual how to draw
        self.cube_transform = transforms.MatrixTransform()
        self.cube.transform = self.cube_transform

        self._timer = app.Timer('auto', connect=self.on_timer, start=True)

#         self.show()

    def on_resize(self, event):
        # Set canvas viewport and reconfigure visual transforms to match.
        vp = (0, 0, self.physical_size[0], self.physical_size[1])
        self.context.set_viewport(*vp)
        self.cube.transforms.configure(canvas=self, viewport=vp)

    def on_draw(self, event):
        gloo.set_viewport(0, 0, *self.physical_size)
        gloo.clear('white', depth=True)

        self.cube.draw()

    def on_timer(self, event):
        self.theta += .5
        self.phi += .5
        self.cube_transform.reset()
        self.cube_transform.rotate(self.theta, (0, 0, 1))
        self.cube_transform.rotate(self.phi, (0, 1, 0))
        self.cube_transform.scale((100, 100, 0.001))
        self.cube_transform.translate((200, 200))
        self.update()

win = Canvas()
win.show()

VispyWidget(height=400, width=400)

In [14]:
# -*- coding: utf-8 -*-
# vispy: gallery 30
# -----------------------------------------------------------------------------
# Copyright (c) Vispy Development Team. All Rights Reserved.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.
# -----------------------------------------------------------------------------
"""
Demonstration of various features of Line visual.
"""

from __future__ import division

import sys
import numpy as np

from vispy import app, gloo, visuals
from vispy.visuals.transforms import STTransform, NullTransform

# vertex positions of data to draw
N = 200
pos = np.zeros((N, 2), dtype=np.float32)
pos[:, 0] = np.linspace(10, 390, N)
pos[:, 1] = np.random.normal(size=N, scale=20, loc=0)

# color array
color = np.ones((N, 4), dtype=np.float32)
color[:, 0] = np.linspace(0, 1, N)
color[:, 1] = color[::-1, 0]

# connection array
connect = np.empty((N-1, 2), np.int32)
connect[:, 0] = np.arange(N-1)
connect[:, 1] = connect[:, 0] + 1
connect[N//2, 1] = N//2  # put a break in the middle


class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, keys='interactive',
                            size=(800, 800))
        # Create several visuals demonstrating different features of Line
        self.lines = [
            # agg-method lines:
            # per-vertex color
            visuals.LineVisual(pos=pos, color=color, method='agg'),
            # solid
            visuals.LineVisual(pos=pos, color=(0, 0.5, 0.3, 1), method='agg'),
            # wide
            visuals.LineVisual(pos=pos, color=color, width=5, method='agg'),

            # GL-method lines:
            visuals.LineVisual(pos=pos, color=color, method='gl'),
            visuals.LineVisual(pos=pos, color=(0, 0.5, 0.3, 1), method='gl'),
            visuals.LineVisual(pos=pos, color=color, width=5, method='gl'),
            # GL-method: "connect" not available in AGG method yet

            # only connect alternate vert pairs
            visuals.LineVisual(pos=pos, color=(0, 0.5, 0.3, 1),
                               connect='segments', method='gl'),
            # connect specific pairs
            visuals.LineVisual(pos=pos, color=(0, 0.5, 0.3, 1),
                               connect=connect, method='gl'),
        ]
        counts = [0, 0]
        for i, line in enumerate(self.lines):
            # arrange lines in a grid
            tidx = (line.method == 'agg')
            x = 400 * tidx
            y = 140 * (counts[tidx] + 1)
            counts[tidx] += 1
            line.transform = STTransform(translate=[x, y])
            # redraw the canvas if any visuals request an update
            line.events.update.connect(lambda evt: self.update())

        self.texts = [visuals.TextVisual('GL', bold=True, font_size=24,
                                         color='w', pos=(200, 40)),
                      visuals.TextVisual('Agg', bold=True, font_size=24,
                                         color='w', pos=(600, 40))]
        for text in self.texts:
            text.transform = NullTransform()
        self.visuals = self.lines + self.texts
        self.show()

    def on_draw(self, event):
        gloo.clear('black')
        for visual in self.visuals:
            visual.draw()

    def on_resize(self, event):
        # Set canvas viewport and reconfigure visual transforms to match.
        vp = (0, 0, self.physical_size[0], self.physical_size[1])
        self.context.set_viewport(*vp)
        for visual in self.visuals:
            visual.transforms.configure(canvas=self, viewport=vp)


if __name__ == '__main__':
    win = Canvas()

    def update(ev):
        pos[:, 1] = np.random.normal(size=N, scale=30, loc=0)
        win.lines[0].set_data(pos)
        win.lines[3].set_data(pos)

    timer = app.Timer()
    timer.connect(update)
    timer.start(1.0)

    if sys.flags.interactive != 1:
        app.run()

VispyWidget(height=800, width=800)

  File "/home/maria/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/maria/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/maria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/maria/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/maria/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 583, in start
    self.io_loop.start()
  File "/home/maria/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/home/maria/anaconda3/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
    self._run_once()
  File "/home/maria/anaconda3/lib/python3.7/asyncio/base_events.py", line 1786, in _run_once
    handle._run()
  File "/home/m

In [21]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------------------------
# Copyright (c) 2015, Vispy Development Team. All Rights Reserved.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.
# -----------------------------------------------------------------------------
"""
This example demonstrates how to draw curved lines (bezier).
"""
 
import sys
 
from vispy import app, gloo, visuals
from vispy.geometry import curves
from vispy.visuals.transforms import STTransform, NullTransform
 
 
class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, title='Bezier lines example',
                            keys='interactive', size=(400, 750))
 
        self.lines = [
            visuals.LineVisual(curves.curve4_bezier(
                (10, 0),
                (50, -190),
                (350, 190),
                (390, 0)
            ), color='w', width=2, method='agg'),
            visuals.LineVisual(curves.curve4_bezier(
                (10, 0),
                (190, -190),
                (210, 190),
                (390, 0)
            ), color='w', width=2, method='agg'),
            visuals.LineVisual(curves.curve3_bezier(
                (10, 0),
                (30, 200),
                (390, 0)
            ), color='w', width=2, method='agg')
        ]
 
        # Translate each line visual downwards
        for i, line in enumerate(self.lines):
            x = 0
            y = 200 * (i + 1)
 
            line.transform = STTransform(translate=[x, y])
 
        self.texts = [
            visuals.TextVisual('Third order curve', bold=True, color='w',
                               font_size=14, pos=(200, 75)),
            visuals.TextVisual('Quadratic curve', bold=True, color='w',
                               font_size=14, pos=(200, 525)),
        ]
 
        for text in self.texts:
            text.transform = NullTransform()
 
        self.visuals = self.lines + self.texts
        self.show()
 
    def on_draw(self, event):
        gloo.clear('black')
        gloo.set_viewport(0, 0, *self.physical_size)
 
        for visual in self.visuals:
            visual.draw()
 
    def on_resize(self, event):
        vp = (0, 0, self.physical_size[0], self.physical_size[1])
        self.context.set_viewport(*vp)
        for visual in self.visuals:
            visual.transforms.configure(canvas=self, viewport=vp)
 
 
if __name__ == '__main__':
    win = Canvas()
 
    if sys.flags.interactive != 1:
        app.run()

VispyWidget(height=750, width=400)

In [22]:
# -*- coding: utf-8 -*-
# vispy: gallery 1
# Copyright (c) 2015, Vispy Development Team.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.
 
"""
Demonstration of Line visual with arbitrary transforms.
 
Several Line visuals are displayed that all have the same vertex position
information, but different transformations.
"""
 
import numpy as np
from vispy import app, gloo, visuals
from vispy.visuals.transforms import (STTransform, LogTransform,
                                      MatrixTransform, PolarTransform)
import vispy.util
vispy.util.use_log_level('debug')
 
# vertex positions of data to draw
N = 200
pos = np.zeros((N, 2), dtype=np.float32)
pos[:, 0] = np.linspace(-350, 350, N)
pos[:, 1] = np.random.normal(size=N, scale=50, loc=0)
 
# One array of colors
color = np.ones((N, 4), dtype=np.float32)
color[:, 0] = np.linspace(0, 1, N)
color[:, 1] = color[::-1, 0]
 
 
class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, keys='interactive', size=(800, 800))
 
        # Define several Line visuals that use the same position data
        # but have different colors and transformations
        colors = [color, (1, 0, 0, 1), (0, 1, 0, 1), (0, 0, 1, 1),
                  (1, 1, 0, 1), (1, 1, 1, 1)]
 
        self.lines = [visuals.LineVisual(pos=pos, color=colors[i])
                      for i in range(6)]
 
        center = STTransform(translate=(400, 400))
 
        self.lines[0].transform = center
 
        self.lines[1].transform = (center *
                                   STTransform(scale=(1, 0.1, 1)))
 
        self.lines[2].transform = (center *
                                   STTransform(translate=(200, 200, 0)) *
                                   STTransform(scale=(0.3, 0.5, 1)))
 
        self.lines[3].transform = (center *
                                   STTransform(translate=(-200, -200, 0),
                                               scale=(200, 1)) *
                                   LogTransform(base=(10, 0, 0)) *
                                   STTransform(translate=(1, 0, 0)))
 
        self.lines[4].transform = MatrixTransform()
        self.lines[4].transform.rotate(45, (0, 0, 1))
        self.lines[4].transform.scale((0.3, 0.3, 1))
        self.lines[4].transform.translate((200, 200, 0))
 
        self.lines[5].transform = (STTransform(translate=(200, 600, 0),
                                               scale=(5, 5)) *
                                   PolarTransform() *
                                   LogTransform(base=(2, 0, 0)) *
                                   STTransform(scale=(0.01, 0.1),
                                               translate=(4, 20)))
 
        self.show()
 
    def on_draw(self, ev):
        gloo.clear('black', depth=True)
        for line in self.lines:
            line.draw()
             
    def on_resize(self, event):
        # Set canvas viewport and reconfigure visual transforms to match.
        vp = (0, 0, self.physical_size[0], self.physical_size[1])
        self.context.set_viewport(*vp)
        for line in self.lines:
            line.transforms.configure(canvas=self, viewport=vp)
 
if __name__ == '__main__':
    win = Canvas()
    import sys
    if sys.flags.interactive != 1:
        app.run()

VispyWidget(height=800, width=800)